In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import geopandas as gpd


from sklearn.linear_model import Ridge, Lasso, LogisticRegression 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_curve, auc, \
precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, RidgeCV, LassoCV, LogisticRegressionCV


import statsmodels.formula.api as smf
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score

import itertools as it

from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image  


import re

from datetime import datetime



In [2]:
train = pd.read_csv('./Datasets/train_regression.csv')
test = pd.read_csv('./Datasets/test_regression.csv')

In [3]:
# making combined df
combined_data = pd.concat([train, test], axis=0, ignore_index=True)

# Clustering the data
lat_long_data = combined_data[['latitude', 'longitude']]
kmeans = KMeans(n_clusters= 8, random_state=10)
combined_data['cluster'] = kmeans.fit_predict(lat_long_data)
cluster_labels = {0: 'Cluster_A', 1: 'Cluster_B', 2: 'Cluster_C',
                  3: 'Cluster_D', 4: 'Cluster_E', 5: 'Cluster_F',6: 'Cluster_G',7: 'Cluster_H'}

combined_data['cluster'] = combined_data['cluster'].map(cluster_labels)


# reseparating
train_data = combined_data.loc[:4999].copy()
test_data = combined_data.loc[5000:].copy()
    
# Helpers


# getting the property
def property_extract(x):
    categories = {
        'condo': ['condo'],
        'hotel': ['hotel'],
        'house': ['home', 'house'],
        'room': ['room']
    }
    for category, keywords in categories.items():
        if any(keyword in x for keyword in keywords):
            return category
    return 'other'

# decding how big each place is
def place_size(x):
    return 's' if x <= 2 else 'm' if x <= 6 else 'l' if x <= 9 else 'xl'

# capping log price
def limit_price(x):
    if x > 7:
        return 7
    else:
        return x

/Users/joelkim/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [4]:
# my cleaning function, it was done in a function in order to regularize the cleaning both the test and training data

def clean_airbnb_data(data):
    
    # nan count, that number of nan values per row
    data['na_count'] = data.isna().sum(axis=1)

    
    # Mapping superhosts to 1 and 0,
    data['host_is_superhost'] = data['host_is_superhost'].map({"t": 1, "f": 0})
    data['host_is_superhost'].fillna(0.0, inplace=True)
    
    # Filling beds with median
    data['beds'].fillna(data['beds'].median(), inplace=True)
    
    
    # Bathrooms text, extracting number
    
    data['bathrooms_text'] = data['bathrooms_text'].str.extract(r'^\s*([\d.]+)').astype(float)
    data['bathrooms_text'].fillna(0, inplace = True)

    
    # Accommodates, normalizing
    data['accommodates'] = data['accommodates'].astype(float)
    
   
    # Has acceptance, mapping
    data['has_availability'] = data['has_availability'].map({"t": 1, "f": 0})
    data['host_acceptance_rate'].fillna(0, inplace=True)

    
    # Profile picture, mapping
    data['host_has_profile_pic'] = data['host_has_profile_pic'].map({"t": 1, "f": 0})
    data['host_has_profile_pic'].fillna(0, inplace=True)
    
    # Identity verified, mapping
    data['host_identity_verified'] = data['host_identity_verified'].map({"t": 1, "f": 0})
    data['host_identity_verified'].fillna(0, inplace=True)

    # Host verifications, getting the number of verifications
    data['host_verifications'] = data['host_verifications'].apply(eval)
    data['host_verifications'] = data['host_verifications'].apply(len)
   

    # Reviews filling in nan with median
    review_columns = ['review_scores_value', 'review_scores_rating', 'review_scores_location', 
                      'review_scores_accuracy', 'reviews_per_month','review_scores_cleanliness',
                      'review_scores_checkin','review_scores_communication']    
    for column in review_columns:
        data[column].fillna(data[column].median(), inplace=True)
    
    
    # availability per day
    
    for days in [30, 60, 90, 365]:
        data[f'availability_{days}'] = data[f'availability_{days}'] / days

    
    data['number_of_reviews_ltm'] = data['number_of_reviews_ltm'] / data['number_of_reviews']
    data['number_of_reviews_l30d'] = data['number_of_reviews_l30d'] / data['number_of_reviews']
    data['number_of_reviews_ltm'].fillna(0, inplace=True)
    data['number_of_reviews_l30d'].fillna(0, inplace=True)
    

   
    
    
    # beds and bathroom per person
    data['bedspp'] = data['beds']/data['accommodates']
    data['bathroomspb'] = data['bathrooms_text']/data['beds']

    data['bathroomspb'].fillna(0, inplace = True)
    data['bedspp'].fillna(0, inplace = True)

    # property type, extracting the property type
    data['property'] = data['property_type'].apply(property_extract)
    
    
    # size, determine the size of each place
    data['size'] = data['accommodates'].apply(place_size)
    

    
    
    # instant bookable, mapping
    data['instant_bookable'] = data['instant_bookable'].map({"t": 1, "f": 0})
    data['instant_bookable'].fillna(0, inplace=True)
    
    # host response rate, extracting the number
    data['host_response_rate'] = data['host_response_rate'].str.rstrip('%').astype('float') / 100.0
    data['host_response_rate'].fillna(data['host_response_rate'].median(), inplace=True)
    
    # acceptance rate, extracting the number
    data['host_acceptance_rate'] = data['host_acceptance_rate'].str.rstrip('%').astype('float') / 100.0
    data['host_acceptance_rate'].fillna(data['host_acceptance_rate'].median(), inplace=True)  
    
    # respone time, ordinality based on recentness
    response_time_mapping = {
    'within an hour': 0,
    'within a few hours': 1,
    'within a day': 2,
    'a few days or more': 3
    }
    
    data['host_response_time'] = data['host_response_time'].map(response_time_mapping)
    data['host_response_time'].fillna(data['host_response_time'].median(), inplace=True)

    # exp transform due to left skwewed
    left_skew_columns = ['review_scores_rating', 'host_acceptance_rate', 'host_response_rate','availability_30','availability_60', 'availability_90', 'availability_365']
    
    for col in left_skew_columns:
         
        data[col] = data[col]**2
        
    

    # host tenure
    data['host_since'] = pd.to_datetime(data['host_since'])

    data['host_tenure_years'] = data['host_since'].apply(lambda x: (pd.Timestamp('2024-03-01') - x).days / 365.25)
    bins = [0, 1, 3, 5, 10, float('inf')]
    labels = ['<1 year', '1-3 years', '3-5 years', '5-10 years', '>10 years']
    data['host_tenure_category'] = pd.cut(data['host_tenure_years'], bins=bins, labels=labels, right=False)

    # host listings
    bins = [0, 1, 5, 10, 20, float('inf')]
    labels = ['1', '2-5', '6-10', '11-20', '>20']
    data['host_listings_count_category'] = pd.cut(data['host_listings_count'], bins=bins, labels=labels, right=False)
    
    
    




# drop outliers, this was found through testing
train_data = train_data.drop([523, 1626, 1823, 2266, 2327, 3129])

# making price normal
train_data['price'] = train_data['price'].replace('[\$,]', '', regex=True).astype(str)
train_data['price'] = pd.to_numeric(train_data['price'], errors='coerce')
train_data['price'] = np.log(train_data['price'])
train_data['price'] = train_data['price'].apply(limit_price)
y_train = train_data.price




clean_airbnb_data(train_data)
clean_airbnb_data(test_data)




In [6]:
categorical_columns = [ 
     "room_type", 
    "cluster", "property", "size", "host_tenure_category", 
    "host_listings_count_category"
]

numerical_columns = [
    "host_response_time", "host_response_rate", 
    "host_acceptance_rate", "host_is_superhost", "host_listings_count", 
    "host_total_listings_count", "host_verifications", "host_has_profile_pic", 
    "host_identity_verified", "accommodates", 
    "bathrooms_text", "beds", "minimum_nights", "maximum_nights", 
    "minimum_minimum_nights", "maximum_minimum_nights", "minimum_maximum_nights", 
    "maximum_maximum_nights", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", 
    "has_availability", "availability_30", "availability_60", "availability_90", 
    "availability_365", "number_of_reviews", "number_of_reviews_ltm", 
    "number_of_reviews_l30d", "review_scores_rating", "review_scores_accuracy", 
    "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", 
    "review_scores_location", "review_scores_value", "instant_bookable", 
    "calculated_host_listings_count", "calculated_host_listings_count_entire_homes", 
    "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms", 
    "reviews_per_month", "na_count", "bedspp", "bathroomspb", "host_tenure_years"
]

all_columns_train = categorical_columns + numerical_columns

X_train = train_data[all_columns_train]
X_test = test_data[all_columns_train]

X_train = pd.get_dummies(X_train, columns=categorical_columns, dtype=int)
X_test = pd.get_dummies(X_test, columns=categorical_columns, dtype=int)

X_train = X_train.drop(columns = ['host_listings_count','host_tenure_years'])
X_test = X_test.drop(columns = ['host_listings_count','host_tenure_years'])

In [17]:
bad_predictors = [
    "cluster_Cluster_H",
    "size_l",
    "host_tenure_category_1-3 years",
    "host_listings_count_category_>20",
    "host_is_superhost",
    "host_response_time",
    "host_identity_verified",
    "cluster_Cluster_D",
    "calculated_host_listings_count_shared_rooms",
    "host_listings_count_category_6-10",
    "property_condo",
    "host_listings_count_category_11-20",
    "cluster_Cluster_B",
    "room_type_Shared room",
    "room_type_Hotel room",
    "has_availability",
    "host_tenure_category_<1 year",
    "cluster_Cluster_G",
    "host_tenure_category_3-5 years",
    "host_has_profile_pic",
    "host_tenure_category_>10 years",
    "host_listings_count_category_1"
]

X_train = X_train.drop(columns = bad_predictors)

In [48]:
X_test = X_test.drop(columns = bad_predictors)

In [22]:
from skopt import BayesSearchCV
from catboost import CatBoostRegressor
from skopt.space import Integer, Real, Categorical
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold

model = CatBoostRegressor(random_state=1, silent=True)

param_grid = {
    'max_depth': Integer(6, 12),
    'learning_rate': Real(0.01, 0.05, 'log-uniform'),
    'reg_lambda': Real(1, 50, 'log-uniform'),
    'n_estimators': Integer(100, 1000),
    'subsample': Real(0.7, 1.0),
    'colsample_bylevel': Real(0.5, 1.0)
}

cv = KFold(n_splits=5,shuffle=True,random_state=1)

bayes_search_cat = BayesSearchCV(
    model, param_grid, cv = cv, n_iter=15, scoring='neg_mean_squared_error', random_state=1, n_jobs=-1, verbose=1
)

bayes_search_cat.fit(X_train, y_train)

print("Best parameters: ", bayes_search_cat.best_params_)
print("Best score: ", bayes_search_cat.best_score_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters:  OrderedDict([('colsample_bylevel', 0.9580115737646444), ('learning_rate', 0.049999999999999996), 

In [36]:
from skopt import BayesSearchCV
from lightgbm import LGBMRegressor
from skopt.space import Integer, Real
from sklearn.model_selection import train_test_split


model = LGBMRegressor(random_state=1, silent=True)

param_grid = {'max_depth': [4,6,8],
              'learning_rate': [0.01, 0.05, 0.1],
               'reg_lambda':[0, 1, 10],
                'n_estimators':[100, 500, 1000],
                'gamma': [0, 10, 100],
                'subsample': [0.5, 0.75, 1.0],
                'colsample_bytree': [0.5, 0.75, 1.0]}

bayes_search_lgbm = BayesSearchCV(
    model, param_grid, n_iter=10, scoring='neg_mean_squared_error', random_state=1, n_jobs=-1, verbose=1
)

bayes_search_lgbm.fit(X_train, y_train)

print("Best parameters: ", bayes_search_lgbm.best_params_)
print("Best score: ", bayes_search_lgbm.best_score_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002052 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2966
[LightGBM] [Info] Number of data points in the train set: 3995, number of used features: 52
[LightGBM] [Info] Start training from score 4.746751
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004706 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2934
[LightGBM] [Info] Number of data points in the train set: 3995, number of used features: 52
[LightGBM] [Info] Start training from score 4.749724
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005406 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2974
[LightGBM] [Info] Number of data points in the train set: 3996, number of used features: 52
[LightGBM] [Info] Start training from score 4.750938
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005424 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3050
[LightGBM] [Info] Number of data points in the train set: 3995, number of used features: 52
[LightGBM] [Info] Start training from score 4.750762
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2962
[LightGBM] [Info] Number of data points in the train set: 3995, number of used features: 52
[LightGBM] [Info] Start training from score 4.753108
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

...

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2974
[LightGBM] [Info] Number of data points in the train set: 3996, number of used features: 52
[Lig

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005240 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2934
[LightGBM] [Info] Number of data points in the train set: 3995, number of used features: 52
[LightGBM] [Info] Start training from score 4.749724
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000679 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3120
[LightGBM] [Info] Number of data points in the train set: 4994, number of used features: 52
[LightGBM] [Info] Start training from score 4.750257
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Best parameters:  OrderedDict([('colsample_bytree', 0.5), ('gamma', 100), ('learning_rate', 0.05), ('max_depth', 8), ('n_estimators', 1000), ('reg_lambda', 1), ('subsample', 1

**Paste the optimal hyperparameter values below.**

Cat: {'learning_rate': 0.049999999999999996, 'loss_function': 'RMSE', 'silent': True, 'subsample': 0.9356582702953511, 'max_depth': 6, 'n_estimators': 1000, 'colsample_bylevel': 0.9580115737646444, 'random_state': 1, 'reg_lambda': 1.41310685787679}

LGBM: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.5, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': 8, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'num_leaves': 31, 'objective': None, 'random_state': 1, 'reg_alpha': 0.0, 'reg_lambda': 1, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'silent': True, 'gamma': 100}

## 3) Model

Using the optimal model hyperparameters, train the model, and paste the code below.

In [44]:
from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold

stacked_model = StackingRegressor(
    estimators=[
        ('cat', best_model_cat),
        ('lgbm', best_model_LGBM)
    ],
    final_estimator=RidgeCV(),
    cv=KFold(n_splits=5, shuffle=True, random_state=1)
)

stacked_model.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011688 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3120
[LightGBM] [Info] Number of data points in the train set: 4994, number of used features: 52
[LightGBM] [Info] Start training from score 4.750257
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('cat',
                               <catboost.core.CatBoostRegressor object at 0x126446620>),
                              ('lgbm',
                               LGBMRegressor(colsample_bytree=0.5, gamma=100,
                                             learning_rate=0.05, max_depth=8,
                                             n_estimators=1000, random_state=1,
                                             reg_lambda=1, silent=True))],
                  final_estimator=RidgeCV())

In [49]:
y_pred = stacked_model.predict(X_test)

y_preds_transformed = np.exp(y_pred)  

y_preds_df = pd.DataFrame(y_preds_transformed, columns=['predicted'])


[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: gamma
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [50]:
test_df = pd.concat([test_data.id.reset_index(drop=True), y_preds_df], axis=1)
test_df.rename(columns={0: 'predicted'}, inplace = True)
test_df.reset_index(inplace = True, drop = True)

In [51]:
test_df.to_csv('predicted_prices_65-2.csv', index = False)